In [2]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''
import os
#from _future_ import print_function
import matplotlib.image as img
import keras
import cv2
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.preprocessing.image import ImageDataGenerator

# train_dir = '/home/temporary/Downloads/train'
# test_dir  = '/home/temporary/Downloads/test'

datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2)
nb_train_samples = 200
nb_test_samples = 50
# batch_size = 25
batch_size = 2
num_classes = 7
epochs = 30
width =300
height = 300

# training_generator = datagen.flow_from_directory(train_dir,
#     target_size=(width, height),
#     batch_size=20,
#     class_mode="categorical")

# testing_generator = datagen.flow_from_directory(test_dir,
#     target_size=(width, height),
#     batch_size=20,
#     class_mode="categorical")


# input image dimensions
# img_rows, img_cols = 28, 28


# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print (x_train.shape)


if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (3, width, height)
else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (width, height, 3)



# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')

# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
saved_weights_name='best_weights.h5'
checkpoint = ModelCheckpoint(saved_weights_name, 
                                     monitor='val_loss', 
                                     verbose=1, 
                                     save_best_only=True, 
                                     mode='min', 
                                     period=1)
tensorboard = TensorBoard(log_dir=os.path.expanduser('~/logs/'), 
                                  histogram_freq=0, 
                                  #write_batch_performance=True,
                                  write_graph=True, 
                                  write_images=False)

model = Sequential()

# model_t = Convolution2D(64, (3, 3), activation='relu')
# model_t.add(Dropout(0.5))
# model_t.add(Dense(num_classes, activation='softmax'))
model.add(keras.applications.inception_v3.InceptionV3(include_top=True,input_tensor=Input(shape=(300,300,3)),weights='imagenet',input_shape=input_shape, pooling=None, classes=1000))


# model_t = Sequential()
# model_t.add(Dense(48, activation='relu'))
# model_t.add(Dropout(0.5))
# model_t.add(Dense(num_classes, activation='softmax'))
#add(Convolution2D(32, kernel_size=(3, 3),
 #                activation='relu',
  #               input_shape=input_shape))

# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(48, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.load_weights('best_weights.h5')
# optimizer = keras.optimizers.Adadelta(lr=0.3)
optimizer = keras.optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

image = img.imread('/home/kandithws/Downloads/pnam.jpg')
resized_image = cv2.resize(image, (300, 300), interpolation=cv2.INTER_CUBIC )
#model.predict()
print(resized_image.shape)
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

(300, 300, 3)


In [53]:
import numpy as np
image = img.imread('/home/kandithws/Downloads/gunsnap1.png')
resized_image = cv2.resize(image, (300, 300), interpolation=cv2.INTER_NEAREST )

img_predict = [None]
img_input = np.full((1,300,300,3),0)
img_input[0,:,:,:] = resized_image
#resized_image = resized_image.reshape(1, 300, 300, 3)
img_input = img_input*1.0/255
print(img_input.shape)
print(model.predict(img_input))


#model.input_shape

os.listdir('/home/kandithws/Downloads/test')

(1, 300, 300, 3)
[[0.11121728 0.03007894 0.02918804 0.06316718 0.03678616 0.03546413
  0.6940983 ]]


['waving_hands',
 'reading',
 'jumping',
 'applaud',
 'drinkin',
 'running',
 'phoning']